<a href="https://colab.research.google.com/github/funkybooboo/CS5060_Program4/blob/main/hw4_Reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4: Reinforcement Learning
## Contributors:
- Brighton Ellis
- Ann Marie Humble
- Nate Stott
- Maddie Patch
___

1. Install stabel baselines3 and Gym

In [ ]:
!pip install stable-baselines3 gymnasium

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO


2. setup the cartPole env and baseline training

In [ ]:
# create environment
env = gym.make("CartPole-v1")

# Train the baseline model
baseline_model = PPO("MlpPolicy", env, verbose=1)
baseline_model.learn(total_timesteps=1000)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.3     |
|    ep_rew_mean     | 21.3     |
| time/              |          |
|    fps             | 296      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------


*Record average baseline reward*

3. Evaluate baseline agent performance

In [ ]:
baseline_rewards = []
episodes = 10

for episode in range(episodes):
    state, _ = env.reset()
    done = False
    episode_reward = 0

    while not done:
        action, _ = baseline_model.predict(state, deterministic=True)
        state, reward, done, _, _ = env.step(action)  # Ensure you're unpacking correctly
        episode_reward += reward

    baseline_rewards.append(episode_reward)

avg_baseline_reward = sum(baseline_rewards) / episodes
print(f"Average reward with baseline model: {avg_baseline_reward}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Average reward with baseline model: 388.7


4. Modify the reward function

In [ ]:
import numpy as np
from gymnasium import RewardWrapper

class CustomCartPoleReward(RewardWrapper):
    def __init__(self, env):
        super(CustomCartPoleReward, self).__init__(env)

    def reward(self, reward):
        # Access the CartPole state and penalize based on the pole's tilt
        x, x_dot, theta, theta_dot = self.env.state
        new_reward = reward - np.abs(theta)  # Penalize for angle from upright
        return new_reward

# Initialize the custom environment
custom_env = CustomCartPoleReward(gym.make("CartPole-v1"))


5. train the agent with modified reward function

In [ ]:
custom_model = PPO("MlpPolicy", custom_env, verbose=1)
custom_model.learn(total_timesteps=10000)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.state to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.state` for environment variables or `env.get_wrapper_attr('state')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23       |
|    ep_rew_mean     | 21.2     |
| time/              |          |
|    fps             | 556      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26          |
|    ep_rew_mean          | 23.8        |
| time/                   |             |
|    fps                  | 410         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008403098 |
|    clip_fraction        | 0.0928      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.000179    |
|    learning_rate        | 0.

6. evaluate the modified agent's performance

In [ ]:
custom_rewards = []
episodes = 10

for episode in range(episodes):
    state, _ = custom_env.reset()
    done = False
    episode_reward = 0

    while not done:
        action, _ = custom_model.predict(state, deterministic=True)
        state, reward, done, _, _ = custom_env.step(action)  # Assuming step returns 4 values
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward with custom reward function: {avg_custom_reward}")


Average reward with custom reward function: 415.7299787929701


Here we can compare `avg_custom_reward` to `avg_baseline_reward` and record observations on how agent's performance was affected

7. modify the model architecture

In [ ]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch as th
from torch import nn

# Custom network architecture
class CustomMLP(nn.Module):
    def __init__(self):
        super(CustomMLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(4, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.network(x)

# Define the model with custom architecture
complex_model = PPO("MlpPolicy", env, policy_kwargs=dict(activation_fn=th.nn.ReLU, net_arch=[128, 128]), verbose=1)
complex_model.learn(total_timesteps=10000)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.9     |
|    ep_rew_mean     | 20.9     |
| time/              |          |
|    fps             | 545      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.1        |
|    ep_rew_mean          | 26.1        |
| time/                   |             |
|    fps                  | 447         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010955214 |
|    clip_fraction        | 0.0968      |
|    clip_range           | 0.2         |
|    entropy_loss  

8. Evaluate the complex model's performance

In [ ]:
complex_rewards = []
episodes = 10

for episode in range(episodes):
    state, _ = env.reset()
    done = False
    episode_reward = 0

    while not done:
        action, _ = complex_model.predict(state, deterministic=True)
        state, reward, done, _, _ = env.step(action)
        episode_reward += reward

    complex_rewards.append(episode_reward)

avg_complex_reward = sum(complex_rewards) / episodes
print(f"Average reward with complex model architecture: {avg_complex_reward}")


Average reward with complex model architecture: 509.1


# HW4: Reinforcement Learning

Due: Fri Nov 15, 2024 11:59pm

30 Possible Points

Group:

	- Ann Marie Humble
	- Brighton Roghelio Ellis
	- Madison Eve Patch
	- Nathaniel Dana Stott

## Summary of Observations

Compare the results from the baseline model, the modified reward function model, and the complex architecture model. Document your observations:

- Baseline Model Average Reward: `avg_baseline_reward`
  - Average over seven executions: 267.3
- Custom Reward Model Average Reward: `avg_custom_reward`
  - Average over seven executions: 508.6
- Complex Model Architecture Average Reward: `avg_complex_reward`
  - Average over seven executions: 439.1

__Based on the average rewards observed, how did each modification affect the agent’s performance?__

From the average rewards observed, we can see that the baseline model did the worst, the complex architecture model did better (with about two times the average reward of the baseline model), and the custom reward model did the best (with about 2.5 times the average reward of the baseline model).

__Why did baseline behave like that?__

With each of these models, our aim is to keep a (simulated) pole balanced atop a cart for as long as possible. The action space has only two options: push the cart to the left, or push the cart to the right. The observation space can see the cart position, cart velocity, pole angle, and pole angular velocity.

Our baseline model accumulates rewards based on how long the pole stays balanced on the cart. The policy must pick paths for the cart that maximize rewards, but since the rewards are vague, the simulation must explore a lot to find which paths work best. The baseline model also updates its policy using the PPO objective, which ensures that only small changes are made to the policy every cycle. This means that the model will take longer to learn, but will hopefully move in a more stable upward direction. PPO also won’t learn between runs, so if you re-run it, you’re starting over from the beginning. All of these factors play into the reasons why our baseline model performed the worst.

__Why did complex architecture behave like that?__

The complex architecture model uses a custom neural network architecture instead of the default architecture used in the baseline model. The complex architecture model has fewer neurons than the baseline model, but deeper layers. This can improve the model’s performance on more complex tasks or environments, but could also lead to slower training and overfitting if the task is too simple. In our case, the complex architecture had better results than our baseline model, so we know that the task wasn’t too simple. However, the complex architecture model did worse than the custom reward model. This makes sense, because the complex model increased its complexity without fine-tuning its rewards.

__Why did custom reward behave like that?__

The custom reward model specifically rewards and penalizes based on the pole’s tilt. Aside from this, the model has the same environment as the baseline model. This is more specific than the baseline reward model, because it tells the model what to do specifically to keep the pole in the air. Even though this model isn’t beefed up (like the complex architecture model is), it is more targeted. Therefore, the model is able to learn faster how to keep the pole in the air, and the rewards are higher.

__Discuss any trends or behaviors noticed in each model setup.__

All model setups we tested had very similar rollout values, with the complex model having the largest. The values for total_timesteps, time_elapsed, and iterations are also the same. The only value that truly differs between models is fps, which is smaller for the custom rewards model, and largest for the complex model.

### Baseline
| Metric              | Value   |
|---------------------|---------|
| **ep_len_mean**     | 20.9    |
| **ep_rew_mean**     | 20.9    |
| **fps**             | 720     |
| **iterations**      | 1       |
| **time_elapsed**    | 2       |
| **total_timesteps** | 2048    |

---

### Custom
| Metric              | Value   |
|---------------------|---------|
| **ep_len_mean**     | 21.5    |
| **ep_rew_mean**     | 19.7    |
| **fps**             | 698     |
| **iterations**      | 1       |
| **time_elapsed**    | 2       |
| **total_timesteps** | 2048    |

---

### Complex
| Metric              | Value   |
|---------------------|---------|
| **ep_len_mean**     | 21.9    |
| **ep_rew_mean**     | 21.9    |
| **fps**             | 729     |
| **iterations**      | 1       |
| **time_elapsed**    | 2       |
| **total_timesteps** | 2048    |

